In [15]:
import pygame
import sys

# Constants
WIDTH, HEIGHT = 600, 600
GRID_SIZE = 8
CELL_SIZE = WIDTH // GRID_SIZE
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 128, 0)


# Initialize the game board
board = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
board[3][4] = board[4][3] = 2  # White
board[3][3] = board[4][4] = 1  # Black

# Current player (1 for White, 2 for Black)


# Initialize Pygame
# # Create the game window

pygame.init()
pygame.font.init() 
my_font = pygame.font.SysFont('Comic Sans MS', 30)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("6x6 Othello")

def draw_board():
    screen.fill(GREEN)
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            pygame.draw.rect(screen, BLACK, (x*CELL_SIZE, y*CELL_SIZE, CELL_SIZE, CELL_SIZE), 1)
            if board[y][x] == 1:
                pygame.draw.circle(screen, WHITE, (x*CELL_SIZE + CELL_SIZE//2, y*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 5)
            elif board[y][x] == 2:
                pygame.draw.circle(screen, BLACK, (x*CELL_SIZE + CELL_SIZE//2, y*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 5)

def is_valid_move(x, y,board,current_player):
    if board[y][x] != 0:
        return False
    directions = [(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1)]
    for dx, dy in directions:
        if check_direction(x, y, dx, dy,board,current_player):
            return True
    return False

def check_direction(x, y, dx, dy,board,current_player):
    opponent = 1 if current_player == 2 else 2
    x, y = x + dx, y + dy
    if not (0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE) or board[y][x] != opponent:
        return False
    while 0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE:
        if board[y][x] == 0:
            return False
        if board[y][x] == current_player:
            return True
        x, y = x + dx, y + dy
    return False

def make_move(x, y,board,current_player):
    board[y][x] = current_player
    directions = [(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1)]
    for dx, dy in directions:
        if check_direction(x, y, dx, dy,board,current_player):
            flip_direction(x, y, dx, dy,board,current_player)

def flip_direction(x, y, dx, dy,board,current_player):
    x, y = x + dx, y + dy
    while board[y][x] != current_player:
        board[y][x] = current_player
        x, y = x + dx, y + dy


In [16]:
def decide_winner(board,current_player):
    black = 0
    white = 0
    for x in board:
        for y in x:
            black += (y == 2)
            white += (y == 1)
    if(black + white != 64) and current_player == 2:
        return 0
    if(black + white != 64) and current_player == 1:
        return 1
    return int(black >= 32)

In [17]:
import random 

def get_move_random(board,current_player):
    valid_moves = []
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            if is_valid_move(x,y,board,current_player): valid_moves.append((x,y))
    if(len(valid_moves) == 0): return None    
    return valid_moves[random.randint(0,len(valid_moves)-1)]

In [18]:
import math
from copy import deepcopy

class Node():
    def __init__(self,x,y,player_who_decides_next):
        self.x = x
        self.y = y
        self.children = []
        self.visits = 0
        self.wins = 0
        self.player = player_who_decides_next
    
    def generate_children(self,board):
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                if is_valid_move(x,y,board,self.player): 
                    if self.player == 1: self.children.append(Node(x,y,2))
                    if self.player == 2: self.children.append(Node(x,y,1))
        if len(self.children) == 0:
            self.children = None

    def select(self,C):
        if(self.player == 2):
            ret = max(self.children,key = lambda child: child.wins/(child.visits+0.001) +  C*( 
            (2.0*math.log(self.visits))/(child.visits+0.001))**0.5)
            return ret
        else:
            ret = max(self.children,key = lambda child: ((child.visits - child.wins)/(child.visits+0.0001) +  C*( 
            (2.0*math.log(self.visits))/(child.visits+0.0001))**0.5))
            return ret

    def rollout(self,depth,C,board): # 1 == simulate from children
        self.visits += 1

        if depth == 0:
            ret = self.simulate(board)
            self.wins += ret
            return ret
        
        
        if self.children == None: 
            ret = decide_winner(board,self.player)
            self.wins += ret
            return ret
        if len(self.children) == 0: self.generate_children(board)
        if self.children == None: 
            ret = decide_winner(board,self.player)
            self.wins += ret
            return ret
        
        
        child = self.select(C)
        make_move(child.x,child.y,board,self.player)
        ret = child.rollout(depth-1,C,board)
        self.wins += ret
        return ret
    
    def simulate(self,board):
        current_player = self.player
        while True:
            move = get_move_random(board,current_player)
            if(move == None): break
            make_move(move[0],move[1],board,current_player)
            if current_player == 2: current_player = 1
            else: current_player = 2
        return decide_winner(board,current_player)
    
    def give_best_move(self,currentplayer):
        if currentplayer == 2 :child = max(self.children,key = lambda child: child.wins/(child.visits+0.001))
        else: child = max(self.children,key = lambda child: (child.visits - child.wins)/(child.visits+0.001))
        return child.x,child.y


def get_move(board,current_player):
    root = Node(-1,-1,current_player)

    for i in range(400):
        sim = deepcopy(board)
        root.rollout(2,1.5,sim)

    return root.give_best_move(current_player)


In [19]:
import time
games = 0
wins = 0
max_games = 50

current_player = 2
while games < max_games:
       
    flag = 0
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            if is_valid_move(x,y,board,current_player): flag=1

    # if current_player == 1 and flag:
    #     x,y = get_move_random(board,current_player)    
    #     if is_valid_move(x, y,board,current_player):
    #         make_move(x,y,board,current_player)
    #     current_player = 2
       
    if current_player == 2 and flag:
        
        grid_x,grid_y = get_move(board,current_player)
        if is_valid_move(grid_x, grid_y,board,current_player):
            make_move(grid_x, grid_y,board,current_player)
           
            current_player = 1

    Quit = False
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            Quit = True
        if event.type == pygame.MOUSEBUTTONDOWN and current_player == 1 and flag:
            x, y = event.pos
            x, y = x // CELL_SIZE, y // CELL_SIZE
            if is_valid_move(x, y,board,current_player):
                make_move(x,y,board,current_player)
                
                current_player = 2
            
    if(Quit): break
        

    if not flag:
        print(games)
        wins += decide_winner(board,current_player)
        board = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        board[3][4] = board[4][3] = 2  # White
        board[3][3] = board[4][4] = 1  # Black
        games += 1
        current_player = 2

    draw_board()
    pygame.display.flip()
    time.sleep(1)
    
pygame.quit()
print("wins ",wins)

wins  0
